In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key, p_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

In [2]:
# Load the csv exported in Part I to a DataFrame

df = pd.read_csv("../WeatherPy/output_data/cities.csv")
print(df)


               City    Lat     Lng  Max Temp  Humidity  Cloudiness  \
0            trelew -43.25  -65.31    64.256        31          45   
1           ushuaia -54.80  -68.30    46.670        65          40   
2           goundam  16.41   -3.67    84.236        62         100   
3        butaritari   3.07  172.79    82.904        74           2   
4           rikitea -23.12 -134.97    73.760        83          24   
..              ...    ...     ...       ...       ...         ...   
550           avera  33.19  -82.53    93.470        44           1   
551  treinta y tres -33.23  -54.38    68.414        86           0   
552            axim   4.87   -2.24    73.940        80          49   
553             shu  43.60   73.76    69.314        41          14   
554          uvalde  29.37  -99.77   100.274        27          40   

     Wind Speed Country        Date  
0          6.80      AR  1596488469  
1          5.70      AR  1596488401  
2          9.89      ML  1596488177  
3      

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = df[["Lat", "Lng"]].astype(float)

# Convert humidity to float and store
humidity = df["Humidity"].astype(float)

In [5]:
# heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights= humidity , 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Narrow down the cities to fit weather conditions.
ideal_vaca = df.loc[(df['Max Temp'] >= 80) & (df['Cloudiness'] <= 20) & (df['Wind Speed'] <= 3)& (df['Humidity'] <= 30)]
ideal_vaca 

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
132,urubicha,-15.38,-62.95,92.516,22,0,1.02,BO,1596488524
175,aripuana,-9.17,-60.63,92.966,28,6,1.45,BR,1596488428
218,coxim,-18.51,-54.76,86.594,19,0,2.29,BR,1596488565
244,laguna,38.42,-121.42,96.278,27,1,2.36,US,1596488575
412,catalina,32.51,-110.92,107.276,16,0,0.89,US,1596488648
431,rio verde de mato grosso,-18.92,-54.84,84.398,23,0,2.26,BR,1596488655
529,pimenta bueno,-11.67,-61.19,93.884,24,0,1.91,BR,1596488488


In [7]:
def get_nearby_locations(params):
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    return requests.get(base_url, params=params).json()



# geocoordinates   
marker_locations = []
hotel_names = []
for idx, row in ideal_vaca.iterrows():
    target_coordinates = f'{row["Lat"]} {row["Lng"]}'
    target_search = 'hotel'
    target_radius = 5000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": p_key
    }
    
    response = get_nearby_locations(params)

    for result in response["results"]:
        try:
            city_object = citipy.nearest_city(row["Lat"], row["Lng"])
            lat_long_set = (result["geometry"]["location"]["lat"], result["geometry"]["location"]["lng"])
            hotel_info = {
                'name': result['name'],
                'locations': lat_long_set,
                'city': city_object.city_name,
                'country': city_object.country_code.upper()
            }
            marker_locations.append(lat_long_set)
            hotel_names.append(hotel_info)
        except:
            continue

markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))

In [8]:
# adding pop up markers with hotel information 

hotel_details = [hotel_name['locations'] for hotel_name in hotel_names]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>city</dt><dd>{city}</dd><dt>country</dt><dd>{country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**hotel_name) for hotel_name in hotel_names]

marker_layer = gmaps.marker_layer(hotel_details, info_box_content= hotel_info)

fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))